In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

In [2]:
from selenium import webdriver

URL = "https://google.com"

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(
    options=options
)

driver.get(URL)
title = driver.title
driver.close()

print(title)

Google


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
!pip install selenium

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

#!apt-get install -y chromium-browser
#!sudo apt install chromium-chromedriver
#!apt-get install chromium-chromedriver

#from selenium import webdriver
#from selenium.webdriver.chrome.options import Options

In [ ]:
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

options.headless = True

driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

In [ ]:
!sudo apt update; sudo apt upgrade

In [ ]:
!sudo apt-get install chromium-bsu

In [15]:
!apt-get install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu0.20.04.3).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [16]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headlines')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dve-shm-uage')

driver = webdriver.Chrome(options=options)

WebDriverException: ignored

In [ ]:
!apt-get update -q
!pip install splinter -q
!pip install selenium -q
!apt install chromium-chromedriver -q

# Load Dependencies
#from collections import Counter
#import nltk
#from nltk.tokenize import word_tokenize
#from nltk.corpus import stopwords
#nltk.download('punkt')
#nltk.download('stopwords')
import pandas as pd
import numpy as np
import os
import time
import requests
from bs4 import BeautifulSoup
#from splinter import Browser
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Disable all warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def print_loading_bar(current, total, time, bar_length=40):
    progress = current / total
    completed_length = int(bar_length * progress)
    remaining_length = bar_length - completed_length

    bar = '█' * completed_length + '-' * remaining_length

    print(f"\r[{bar}] Processing {current} of {total} COMPLETE {progress*100:.2f}% Time Lapsed {time:.2f} Seconds", end ="")

def load_join(select, loaded):
    selected_df = pd.read_csv("https://www.dropbox.com/s/" + select + "?dl=1", index_col="review_index")
    if loaded is not None:
        selected_df = pd.merge(loaded, selected_df, on='review_index')
    return selected_df

In [ ]:
load_metadata = True
load_ratings = False
load_opinions = False
load_descriptions = False
load_world_happiness_report = True
word_counted_preprocessing_bypass = False

index_df = None
if load_metadata == True:
    load_index = "goqyfyeu2qvpsmo/metadata_df.csv"
    index_df = load_join(load_index, index_df)
if load_ratings == True:
    load_index = "9j2j86xwqrmrljx/ratings_df.csv"
    index_df = load_join(load_index, index_df)
if load_opinions == True:
    load_index = "q8v9f6rbb4z12df/opinion_df.csv"
    index_df = load_join(load_index, index_df)
if load_descriptions == True:
    load_index = "hnp8sebleh6dzgt/descriptions_df.csv"
    index_df = load_join(load_index, index_df)

if load_world_happiness_report == True:
    WHR_df = pd.read_csv("https://www.dropbox.com/s/jyr4e7fleevrb1s/WHR2023.csv?dl=1")

if word_counted_preprocessing_bypass == True:
    joint_df = pd.read_csv("https://www.dropbox.com/s/e64gm79gxu1tgip/joint_df.csv?dl=1")

In [ ]:
working_df = index_df.copy()
working_WHR_df = WHR_df.copy()

In [ ]:
working_firm_column = working_df['firm'].copy()
working_firm_column.value_counts()

IBM                         60436
McDonald-s                  49450
Deloitte                    46995
EY                          34050
PwC                         33227
                            ...  
i-Net-Solution                  4
Health-Protection-Agency        4
The-Survey-Association          3
NPSA                            2
UKCIL                           1
Name: firm, Length: 428, dtype: int64

In [ ]:
def search_business_industry(business_name):
    url = f"https://www.glassdoor.com/Search/results.htm?keyword={business_name}"

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run Chrome in headless mode

    driver_path = "/path/to/chromedriver"  # Replace with the actual path to chromedriver executable
    driver = webdriver.Chrome(options=options, executable_path=driver_path)
    driver.get(url)

    html = driver.page_source
    driver.quit()

    return html

test = search_business_industry("EY")
print(test)

TypeError: ignored

In [ ]:
def search_business_industry(business_name):
    # Set the path to the Chromium browser driver
    chrome_driver_path = "/usr/lib/chromium-browser/chromedriver"

    # Configure the Chrome options
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument(f"--chromedriver-executable={chrome_driver_path}")

    # Create a new Chrome driver instance
    browser = Browser("chrome", options=chrome_options)

    # Construct the search query
    url = f"https://www.glassdoor.com/Search/results.htm?keyword={business_name}"

    # Visit the URL
    browser.visit(url)

    # Wait for the page to load
    time.sleep(2)

    # Get the page source
    html = browser.html

    # Parse the HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')

    # Close the browser
    browser.quit()

    return soup

test = search_business_industry("EY")
print(test)

WebDriverException: ignored